# *Скрейпване на laptopbg*

### *Нужни библиотеки*

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### *Построяване на първоначалния Soup object*

In [2]:
base_url = 'https://laptop.bg/desktop_video_cards-all'
headers = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(base_url, headers = headers)
soup = BeautifulSoup(response.content, 'lxml')

### *Вземане на броя на страниците, които ще скрейпваме*

In [3]:
pages_div = soup.find('div', class_ = 'pagination')
num_pages = pages_div.contents[-2].get_text()
pages = int(num_pages)
print(pages)

7


### *Скрейпване на всички страници и записване на информацията в list*

In [4]:
product_data = []
for num in range(1, pages+1):
    target_url = f'{base_url}?page={num}'
    response = requests.get(target_url, headers = headers)
    soup = BeautifulSoup(response.content, 'lxml')
    article = soup.find_all('article', itemtype = 'http://schema.org/product')
    for art in article:
        sku = art.find('meta', itemprop = 'sku').get('content')
        div_price = art.find('div', class_ = 'price-container')
        price = div_price.find('span', itemprop = 'price').get_text()
        product_data.append({
            'SKU' : sku,
            'Price_laptopbg' : price
        })

### Създаване на DataFrame от данните

In [5]:
df = pd.DataFrame(product_data)
df.head(10)

,SKU,Price_laptopbg
0,RTX_3050_VENTUS_2X_XS_8G_OC,449
1,RX6600 CLD 8G,449
2,ASR-VC-RX6600-CLW-8G,535
3,RTX_3060_VENTUS_2X_12G_OC,619
4,RTX_4060_VENTUS_2X_BLACK_8G_OC,639
5,B580 SL 12GO,639
6,RTX_4060_VENTUS_2X_WHITE_8G_OC,645
7,ASR-VC-RX7600-SL-8GO,649
8,90YV0JC7-M0NA00,659
9,G5060-8V2CW,719


### *Записване на данните в .csv формат*

In [6]:
df.to_csv('laptopbg_gpu_sku_prices.csv', index = False)